In [53]:
%pip install scikit-image


  Using cached scikit_image-0.23.2-cp312-cp312-win_amd64.whl.metadata (14 kB)
Using cached scikit_image-0.23.2-cp312-cp312-win_amd64.whl (12.8 MB)
Note: you may need to restart the kernel to use updated packages.


# Imports

In [124]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.spatial import distance
from collections import Counter

# Initial Testing

In [153]:
# img = cv2.imread('data/R200/R200f_mandela.jpg',cv2.IMREAD_COLOR)
# img2 = cv2.imread('data/R100/R100b_mandela5.jpg',cv2.IMREAD_COLOR)
# #img_blur = cv2.GaussianBlur(img, (5,5), 0)
# img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img_gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# #img_eq = cv2.equalizeHist(img_gray)
# #img_neg = cv2.bitwise_not(img_eq)
# img_resized = cv2.resize(img_gray, (574,265))
# img_resized2 = cv2.resize(img_gray2, (574,265))



orb = cv2.ORB_create(nfeatures=1000)
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']


def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_blurred = cv2.GaussianBlur(img, (3, 3), 0)
    img_gray = cv2.cvtColor(img_blurred, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_resized = cv2.resize(img_eq, (574, 265))
    #adaptive thresholding
    img2 = cv2.adaptiveThreshold(img_resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
    
    

    return img_gray

def extract_features(img):
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return descriptors

# Process all images and extract features
train_descriptors = []
train_labels = []
for label in classes:
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(class_dir, filename)
            preprocessed_img = preprocess_image(img_path)
            descriptors = extract_features(preprocessed_img)
            if descriptors is not None:
                train_descriptors.append(descriptors)
                train_labels.append(label)

# Preprocess and extract features from the test image
test_img_path = 'test20_2.jpg'
test_img = preprocess_image(test_img_path)
test_descriptors = extract_features(test_img)

print(test_descriptors.shape)

# Use BFMatcher to match the descriptors to the test image
bf = cv2.BFMatcher()
all_matches = []

for i, train_desc in enumerate(train_descriptors):
    matches = bf.knnMatch(test_descriptors, train_desc, k=2)
    # Apply ratio test
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    all_matches.extend([(train_labels[i], m) for m in good_matches])

k = 23  # Number of nearest neighbors to consider for the final classification
all_matches.sort(key=lambda x: x[1].distance)
top_k_matches = all_matches[:k]

# Count the labels of the top k good matches
matched_labels = [label for label, match in top_k_matches]
most_common_label = Counter(matched_labels).most_common(1)[0][0]

print(f'The predicted class for the input image is: {most_common_label}')
# show preprocessed image
cv2.imshow('image', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()







(924, 32)
The predicted class for the input image is: R20


# Testing in Code Cleaned

In [155]:
import cv2
import os
import numpy as np
from collections import Counter

# Constants and settings
orb = cv2.ORB_create(nfeatures=1000)
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']
k = 23  # Number of nearest neighbors to consider for the final classification

# Function to preprocess images
def preprocess_image(img):
    img_blurred = cv2.GaussianBlur(img, (3, 3), 0)
    img_gray = cv2.cvtColor(img_blurred, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_resized = cv2.resize(img_eq, (574, 265))
    # Adaptive thresholding
    img_thresh = cv2.adaptiveThreshold(img_resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return img_gray

# Function to extract features using ORB
def extract_features(img):
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return descriptors

# Function to load training data
def load_training_data(data_dir, classes):
    train_descriptors = []
    train_labels = []
    for label in classes:
        class_dir = os.path.join(data_dir, label)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(class_dir, filename)
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                preprocessed_img = preprocess_image(img)
                descriptors = extract_features(preprocessed_img)
                if descriptors is not None:
                    train_descriptors.append(descriptors)
                    train_labels.append(label)
    return train_descriptors, train_labels

# Function to classify an image based on extracted features
def classify_image(test_descriptors, train_descriptors, train_labels, k=23):
    bf = cv2.BFMatcher()
    all_matches = []
    for i, train_desc in enumerate(train_descriptors):
        matches = bf.knnMatch(test_descriptors, train_desc, k=2)
        # Apply ratio test
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        all_matches.extend([(train_labels[i], m) for m in good_matches])
    
    all_matches.sort(key=lambda x: x[1].distance)
    top_k_matches = all_matches[:k]

    matched_labels = [label for label, match in top_k_matches]
    most_common_label = Counter(matched_labels).most_common(1)[0][0]
    return most_common_label

# Function to classify a single test image
def classify_test_image(test_img_path):
    train_descriptors, train_labels = load_training_data(data_dir, classes)
    
    test_img = cv2.imread(test_img_path, cv2.IMREAD_COLOR)
    preprocessed_img = preprocess_image(test_img)
    test_descriptors = extract_features(preprocessed_img)
    
    if test_descriptors is not None:
        predicted_class = classify_image(test_descriptors, train_descriptors, train_labels, k)
        print(f'The predicted class for the input image is: {predicted_class}')
        # Show preprocessed image with the classification result
        cv2.putText(preprocessed_img, f'Class: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Preprocessed Image', preprocessed_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print('No descriptors found in the test image.')

if __name__ == "__main__":
    test_img_path = 'test20_2.jpg'  # Path to the test image
    classify_test_image(test_img_path)


The predicted class for the input image is: R20


# TKINTER IMPLEMENTATION

In [3]:
import cv2
import os
import numpy as np
from collections import Counter
from tkinter import Tk, Label, Button, filedialog, StringVar
from PIL import Image, ImageTk

# Constants and settings
orb = cv2.ORB_create(nfeatures=1000)
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']
k = 23  # Number of nearest neighbors to consider for the final classification

# Function to preprocess images
def preprocess_image(img):
    img_blurred = cv2.GaussianBlur(img, (3, 3), 0)
    img_gray = cv2.cvtColor(img_blurred, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_resized = cv2.resize(img_eq, (574, 265))
    # Adaptive thresholding
    img_thresh = cv2.adaptiveThreshold(img_resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return img_gray

# Function to extract features using ORB
def extract_features(img):
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return descriptors

# Function to load training data
def load_training_data(data_dir, classes):
    train_descriptors = []
    train_labels = []
    for label in classes:
        class_dir = os.path.join(data_dir, label)
        for filename in os.listdir(class_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(class_dir, filename)
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                preprocessed_img = preprocess_image(img)
                descriptors = extract_features(preprocessed_img)
                if descriptors is not None:
                    train_descriptors.append(descriptors)
                    train_labels.append(label)
    return train_descriptors, train_labels

# Function to classify an image based on extracted features
def classify_image(test_descriptors, train_descriptors, train_labels, k=23):
    bf = cv2.BFMatcher()
    all_matches = []
    for i, train_desc in enumerate(train_descriptors):
        matches = bf.knnMatch(test_descriptors, train_desc, k=2)
        # Apply ratio test
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        all_matches.extend([(train_labels[i], m) for m in good_matches])
    
    all_matches.sort(key=lambda x: x[1].distance)
    top_k_matches = all_matches[:k]

    matched_labels = [label for label, match in top_k_matches]
    most_common_label = Counter(matched_labels).most_common(1)[0][0]
    return most_common_label

# Function to classify a single test image
def classify_test_image(test_img_path, result_var):
    train_descriptors, train_labels = load_training_data(data_dir, classes)
    
    test_img = cv2.imread(test_img_path, cv2.IMREAD_COLOR)
    preprocessed_img = preprocess_image(test_img)
    test_descriptors = extract_features(preprocessed_img)
    
    if test_descriptors is not None:
        predicted_class = classify_image(test_descriptors, train_descriptors, train_labels, k)
        result_var.set(f'The predicted class for the input image is: {predicted_class}')
    else:
        result_var.set('No descriptors found in the test image.')

fileLocation = ''
img_label = None
# Tkinter application
def browse_image():
    filename = filedialog.askopenfilename()
    if filename:
        global fileLocation
        fileLocation = filename
        classify_test_image(filename, result_var)
        display_image()

def display_image():
    global img_label
    img = Image.open(fileLocation)
    img = img.resize((300, 150), Image.Resampling.LANCZOS)
    img = ImageTk.PhotoImage(img)
    img_label = Label(image=img)
    img_label.image = img
    img_label.pack()


if __name__ == "__main__":
    # Set up Tkinter
    root = Tk()
    root.title("Image Classifier")
    root.geometry("400x300")

    
    result_var = StringVar()
    result_var.set("No image selected")

    browse_button = Button(root, text="Browse Image", command=browse_image)
    browse_button.pack()
    

    result_label = Label(root, textvariable=result_var)
    result_label.pack()

    root.mainloop()
